### Description:

Goes through each node and attempts to grab its tiles from Google Street Maps API (grabs two tiles that will later become the panoramic)


### API Setup:

In [ ]:
from dotenv import load_dotenv
import requests
import os

API_KEY = os.getenv("API_KEY")
SESSION_ID = ""

def setup_session():
    global SESSION_ID
    session_url = f"https://tile.googleapis.com/v1/createSession?key={API_KEY}"

    payload = {
        "mapType": "streetview",
        "language": "en-US",
        "region": "US"
    }
    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(session_url, json=payload, headers=headers)

    # Print the response
    if response.status_code == 200:
        # print("Session Token Created:", response.json())
        SESSION_ID = response.json()['session']
    else:
        print("Error:", response.status_code, response.text)

setup_session()

### API Functions

In [ ]:
# gets the image for the panoId, the panorama automatically faces the direction of traggic (in the center
def get_image_for_panoId(pano_id, output_path, tile_x=0, tile_y=0, z=1):
    url = f"https://tile.googleapis.com/v1/streetview/tiles/{z}/{tile_x}/{tile_y}?session={SESSION_ID}&key={API_KEY}&panoId={pano_id}&zoom=1"

    response = requests.get(url)
    # Print the response
    if response.status_code == 200:
       with open(output_path, "wb") as file:
            file.write(response.content)
    else:
        print("Error:", response.status_code, response.text)


def get_data_from_cords(lat, long, radius=10):
    url = f"https://tile.googleapis.com/v1/streetview/metadata?session={SESSION_ID}&key={API_KEY}&lat={lat}&lng={long}&radius={radius}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)


def get_data_from_panoId(pano_id):
    url = f"https://maps.googleapis.com/maps/api/streetview/metadata?pano={pano_id}&key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)

### Functions

In [ ]:
import numpy as np
from PIL import Image

def convert_heading_to_anticlockwise_from_east(heading):
    # Convert the heading to anti-clockwise from east
    heading = 90 - heading
    if heading < 0:
        heading += 360

    return heading

# Removes all rows of black pixels from the bottom of an image.
def remove_black_rows(image):
  
    image_array = np.array(image)
    
    # Check for black rows at the bottom
    is_black_row = np.all(image_array == 0, axis=(1, 2))
    
    # Find the last non-black row
    last_non_black_row = np.where(is_black_row == False)[0][-1]
    
    # Slice the image array to remove black rows at the bottom
    cropped_image_array = image_array[:last_non_black_row + 1]
    
    # Convert back to an image
    cropped_image = Image.fromarray(cropped_image_array)
    return cropped_image

def crop_both_tile_images(tile_path_0, tile_path_1, coord_data):

    # remove black rows from first image, and save
    with Image.open(tile_path_0) as img:
        # first remove potential black/blank rows at the bottom
        img = remove_black_rows(img)
        img.save(tile_path_0)
        # plot_image(img, "no_black_space "+tile_path_0 )


    # remove black rows from second image, crop it to the right, so panorama width is 2x height, and save
    with Image.open(tile_path_1) as img:
        img = remove_black_rows(img)
        width_to_crop = 2 * (img.width - img.height)

        top = 0
        left = 0
        right = img.width - width_to_crop
        bottom = img.height

        cropped_img = img.crop((left, top, right, bottom))


        # plot_image(img, "no_black_space "+tile_path_1 )
        # plot_image(cropped_img, "cropped_" +tile_path_1 )

        cropped_img.save(tile_path_1)

In [ ]:
def combine_panoramic_tiles(tile1_path, tile2_path, output_path):

    # Open the two image tiles
    tile1 = Image.open(tile1_path)
    tile2 = Image.open(tile2_path)

    # Create a new blank image with combined width and same height
    combined_width = tile1.size[0] + tile2.size[0]
    combined_height = tile1.size[1]
    combined_image = Image.new("RGB", (combined_width, combined_height))

    # Paste the tiles side by side
    combined_image.paste(tile1, (0, 0))  # Place tile1 at the left
    combined_image.paste(tile2, (tile1.size[0], 0))  # Place tile2 to the right of tile1

    # Save the combined image
    combined_image.save(output_path)
    print(f"Combined panorama saved to {output_path}")


# 
def get_save_images_for_coords(lat, long):

    # first get tile images
    coord_data = get_data_from_cords(lat, long)
    pano_id =  coord_data['panoId']
    heading = convert_heading_to_anticlockwise_from_east(coord_data['heading'])
    tilt = coord_data['tilt'] - 90

    # TODO store some data about this panoramic image (like heading & tilt)

    filename_1 = f"data/tile_imgs/{pano_id}_1.jpg"
    filename_2 = f"data/tile_imgs/{pano_id}_2.jpg"
    combined_filename = f"data/panoramic_imgs/{pano_id}.jpg"

    # gets then stores the images
    get_image_for_panoId(pano_id, filename_1, 0, 0)
    get_image_for_panoId(pano_id, filename_2, 1, 0)

    # some images are not squares, so we need to crop those ones into squares
    crop_both_tile_images(filename_1, filename_2, coord_data)

    # then create panoramic image
    combine_panoramic_tiles(filename_1, filename_2, combined_filename)





### Main

In [ ]:
def main():
    # TODO loop through all nodes can get their panoramic tiles
    get_save_images_for_coords(38.8898288,-76.9968158)




main()